#video classification (1) of COVID-US (data preparation)

https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/

**preprocessed(clean) videos -> split -> pre-trained CNN** -> LSTM or GRU -> dense layer -> result(accuracy)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/COVID-US-main/COVID-US-main

/content/drive/MyDrive/COVID-US-main/COVID-US-main


In [6]:
route = '/content/drive/MyDrive/COVID-US-main/COVID-US-main/'

##0. Import libraries

In [7]:
import cv2 # for capturing videos
import math # for mathematical operations
%matplotlib inline
import matplotlib.pyplot as plt # for plotting the images
import numpy as np # for mathematical operations
import pandas as pd
import tensorflow as tf

from skimage.transform import resize # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

import keras
from keras.utils import np_utils
from keras.preprocessing import image # for preprocessing the images
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.layers import TimeDistributed, GRU, LSTM, GlobalAveragePooling1D

from scipy import stats as s
from sklearn.metrics import accuracy_score

import os
import time
import random
import PIL
from PIL import Image
# import re
# import shutil

In [8]:
frames=14
width=224
height=224
channels=3
train_test_ratio=0.3
train_val_ratio=0.2

##1. Prepare preprocessd(clean) videos

In [9]:
video_dataset = pd.read_csv("utils/video_files_clean_properties_addition.csv")
video_dataset

,filename,framerate,width,height,frame_count,duration_secs,disease,source
0,1_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,64.0,4.27,0,1
1,2_butterfly_covid_prc_convex_clean.avi,15.0,624.0,624.0,158.0,10.53,0,1
2,3_butterfly_covid_prc_convex_clean.avi,15.0,1054.0,1054.0,90.0,6.00,0,1
3,4_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,108.0,7.20,0,1
4,5_butterfly_covid_prc_convex_clean.avi,15.0,810.0,810.0,249.0,16.60,0,1
...,...,...,...,...,...,...,...,...
237,239_paper_normal_prc_convex_clean.avi,15.0,512.0,512.0,197.0,13.13,2,8
238,240_paper_other_prc_convex_clean.avi,15.0,512.0,512.0,96.0,6.40,2,8
239,241_paper_other_prc_convex_clean.avi,15.0,512.0,512.0,98.0,6.53,2,8
240,242_paper_other_prc_convex_clean.avi,15.0,512.0,512.0,299.0,19.93,2,8


In [10]:
video_dataset=video_dataset[video_dataset['filename'].str.contains('covid|pneumonia|normal')]
video_dataset=video_dataset[video_dataset['filename'].str.contains('convex')]

video_dataset['disease'].value_counts()

0    63
1    40
2    19
Name: disease, dtype: int64

In [11]:
normal_dataset = video_dataset[video_dataset['filename'].str.contains('normal')]
normal_dataset.reset_index(drop = True)
normal_dataset

,filename,framerate,width,height,frame_count,duration_secs,disease,source
19,20_butterfly_normal_prc_convex_clean.avi,15.0,594.0,594.0,142.0,9.47,2,1
20,21_butterfly_normal_prc_convex_clean.avi,15.0,820.0,820.0,99.0,6.60,2,1
34,36_grepmed_normal_prc_convex_clean.avi,15.0,540.0,540.0,85.0,5.67,2,2
60,62_pocusatlas_normal_prc_convex_clean.avi,15.0,376.0,376.0,60.0,4.00,2,4
69,71_pocusatlas_normal_prc_convex_clean.avi,15.0,450.0,450.0,59.0,3.93,2,4
74,76_pocusatlas_normal_prc_convex_clean.avi,15.0,338.0,338.0,60.0,4.00,2,4
75,77_pocusatlas_normal_prc_convex_clean.avi,15.0,236.0,236.0,60.0,4.00,2,4
150,152_radio_normal_prc_convex_clean.avi,15.0,270.0,270.0,131.0,8.73,2,5
173,175_uf_normal_prc_convex_clean.avi,15.0,480.0,480.0,38.0,2.53,2,7
174,176_uf_normal_prc_convex_clean.avi,15.0,512.0,512.0,53.0,3.53,2,7


In [12]:
covid_dataset = video_dataset[video_dataset['filename'].str.contains('covid')]
covid_dataset.reset_index(drop = True)
covid_dataset

,filename,framerate,width,height,frame_count,duration_secs,disease,source
0,1_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,64.0,4.27,0,1
1,2_butterfly_covid_prc_convex_clean.avi,15.0,624.0,624.0,158.0,10.53,0,1
2,3_butterfly_covid_prc_convex_clean.avi,15.0,1054.0,1054.0,90.0,6.00,0,1
3,4_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,108.0,7.20,0,1
4,5_butterfly_covid_prc_convex_clean.avi,15.0,810.0,810.0,249.0,16.60,0,1
...,...,...,...,...,...,...,...,...
202,204_paper_covid_prc_convex_clean.avi,15.0,680.0,680.0,235.0,15.67,0,8
203,205_paper_covid_prc_convex_clean.avi,15.0,680.0,680.0,234.0,15.60,0,8
216,218_clarius_covid_prc_convex_clean.avi,15.0,400.0,400.0,104.0,6.93,0,9
218,220_clarius_covid_prc_convex_clean.avi,15.0,400.0,400.0,118.0,7.87,0,9


In [13]:
pneumonia_dataset = video_dataset[video_dataset['filename'].str.contains('pneumonia')]
pneumonia_dataset.reset_index(drop = True)
pneumonia_dataset

,filename,framerate,width,height,frame_count,duration_secs,disease,source
23,25_grepmed_pneumonia_prc_convex_clean.avi,15.0,664.0,664.0,300.0,20.00,1,2
25,27_grepmed_pneumonia_prc_convex_clean.avi,15.0,344.0,344.0,91.0,6.07,1,2
32,34_grepmed_pneumonia_prc_convex_clean.avi,15.0,550.0,550.0,458.0,30.53,1,2
35,37_grepmed_pneumonia_prc_convex_clean.avi,15.0,652.0,652.0,187.0,12.47,1,2
36,38_grepmed_pneumonia_prc_convex_clean.avi,15.0,540.0,540.0,300.0,20.00,1,2
38,40_grepmed_pneumonia_prc_convex_clean.avi,15.0,354.0,354.0,114.0,7.60,1,2
39,41_grepmed_pneumonia_prc_convex_clean.avi,15.0,386.0,386.0,151.0,10.07,1,2
41,43_litfl_pneumonia_prc_convex_clean.avi,15.0,540.0,540.0,115.0,7.67,1,3
42,44_litfl_pneumonia_prc_convex_clean.avi,15.0,540.0,540.0,18.0,1.20,1,3
43,45_litfl_pneumonia_prc_convex_clean.avi,15.0,540.0,540.0,21.0,1.40,1,3


In [31]:
import shutil
files = glob('data/video/clean_copy2/*')
for f in files:
    os.remove(f)
os.rmdir("data/video/clean_copy2")
shutil.copytree("data/video/clean", "data/video/clean_copy2")

'data/video/clean_copy2'

In [32]:
normal_aug_file_list = []
SEED_0 = 777
SEED_1 = 7777
SEED_2 = 77777
SEED_3 = 777777
angle = 15
for each_video in normal_dataset['filename']:
    cap = cv2.VideoCapture('data/video/clean/' + each_video)
    frameFPS = cap.get(cv2.CAP_PROP_FPS)
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*"DIVX") # codec
    filename = 'data/video/clean_copy2/' + each_video
    index = filename.find('.avi')
    aug_0_filename = filename[:index] + '_aug_0' + filename[index:]
    aug_1_filename = filename[:index] + '_aug_1' + filename[index:]
    aug_2_filename = filename[:index] + '_aug_2' + filename[index:]
    aug_3_filename = filename[:index] + '_aug_3' + filename[index:]
    aug_4_filename = filename[:index] + '_aug_4' + filename[index:]
    normal_aug_file_list.append(aug_0_filename.split('/')[-1])
    normal_aug_file_list.append(aug_1_filename.split('/')[-1])
    normal_aug_file_list.append(aug_2_filename.split('/')[-1])
    normal_aug_file_list.append(aug_3_filename.split('/')[-1])
    normal_aug_file_list.append(aug_4_filename.split('/')[-1])
    videoWriter_0 = cv2.VideoWriter(aug_0_filename, fourcc, frameFPS, (frameWidth, frameHeight))
    videoWriter_1 = cv2.VideoWriter(aug_1_filename, fourcc, frameFPS, (frameWidth, frameHeight))
    videoWriter_2 = cv2.VideoWriter(aug_2_filename, fourcc, frameFPS, (frameWidth, frameHeight))
    videoWriter_3 = cv2.VideoWriter(aug_3_filename, fourcc, frameFPS, (frameWidth, frameHeight))
    videoWriter_4 = cv2.VideoWriter(aug_4_filename, fourcc, frameFPS, (frameWidth, frameHeight))

    while (cap.isOpened()):
        ret, frame = cap.read()
        if frame is None:
          break
        frame_0 = cv2.flip(frame,1)
        videoWriter_0.write(frame_0)

        random.seed(SEED_0)
        rand_angle = int(random.uniform(-angle,angle))
        h, w = frame.shape[:2]
        M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), rand_angle, 1)
        frame_1 = cv2.warpAffine(frame, M, (w, h))
        frame_1 = np.uint8(frame_1)
        videoWriter_1.write(frame_1)
        
        random.seed(SEED_1)
        rand_angle = int(random.uniform(-angle,angle))
        h, w = frame.shape[:2]
        M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), rand_angle, 1)
        frame_2 = cv2.warpAffine(frame, M, (w, h))
        frame_2 = np.uint8(frame_2)
        videoWriter_2.write(frame_2)

        random.seed(SEED_2)
        rand_angle = int(random.uniform(-angle,angle))
        h, w = frame.shape[:2]
        M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), rand_angle, 1)
        frame_3 = cv2.warpAffine(frame, M, (w, h))
        frame_3 = np.uint8(frame_3)
        videoWriter_3.write(frame_3)

        random.seed(SEED_3)
        rand_angle = int(random.uniform(-angle,angle))
        h, w = frame.shape[:2]
        M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), rand_angle, 1)
        frame_4 = cv2.warpAffine(frame, M, (w, h))
        frame_4 = np.uint8(frame_4)
        videoWriter_4.write(frame_4)
  
    cap.release()
    videoWriter_0.release()
    videoWriter_1.release()
    videoWriter_2.release()
    videoWriter_3.release()
    videoWriter_4.release()

    SEED_0 = SEED_0+1
    SEED_1 = SEED_1+1
    SEED_2 = SEED_2+1
    SEED_3 = SEED_3+1

KeyboardInterrupt: ignored

In [16]:
covid_aug_file_list = []
for each_video in covid_dataset['filename']:
    cap = cv2.VideoCapture('data/video/clean/' + each_video)
    frameFPS = cap.get(cv2.CAP_PROP_FPS)
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*"DIVX") # codec
    filename = 'data/video/clean_copy2/' + each_video
    index = filename.find('.avi')
    aug_0_filename = filename[:index] + '_aug_0' + filename[index:]
    covid_aug_file_list.append(aug_0_filename.split('/')[-1])
    videoWriter_0 = cv2.VideoWriter(aug_0_filename, fourcc, frameFPS, (frameWidth, frameHeight))

    while (cap.isOpened()):
        ret, frame = cap.read()
        frame_0 = cv2.flip(frame,1)
        if frame is None:
            break
        videoWriter_0.write(frame_0)
  
    cap.release()
    videoWriter_0.release()

In [17]:
pneumonia_aug_file_list = []
SEED = 888
angle = 15
for each_video in pneumonia_dataset['filename']:
    cap = cv2.VideoCapture('data/video/clean/' + each_video)
    frameFPS = cap.get(cv2.CAP_PROP_FPS)
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*"DIVX") # codec
    filename = 'data/video/clean_copy2/' + each_video
    index = filename.find('.avi')
    aug_0_filename = filename[:index] + '_aug_0' + filename[index:]
    aug_1_filename = filename[:index] + '_aug_1' + filename[index:]
    pneumonia_aug_file_list.append(aug_0_filename.split('/')[-1])
    pneumonia_aug_file_list.append(aug_1_filename.split('/')[-1])
    videoWriter_0 = cv2.VideoWriter(aug_0_filename, fourcc, frameFPS, (frameWidth, frameHeight))
    videoWriter_1 = cv2.VideoWriter(aug_1_filename, fourcc, frameFPS, (frameWidth, frameHeight))

    while (cap.isOpened()):
        ret, frame = cap.read()
        if frame is None:
          break
        frame_0 = cv2.flip(frame,1)
        videoWriter_0.write(frame_0)

        random.seed(SEED)
        rand_angle = int(random.uniform(-angle,angle))
        h, w = frame.shape[:2]
        M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), rand_angle, 1)
        frame_1 = cv2.warpAffine(frame, M, (w, h))
        frame_1 = np.uint8(frame_1)
        videoWriter_1.write(frame_1)
        SEED = SEED + 1
  
    cap.release()
    videoWriter_0.release()
    videoWriter_1.release()

    SEED = SEED + 1

In [18]:
normal_aug_dataset = pd.DataFrame()
normal_normal_dataset = pd.concat([normal_dataset, normal_dataset, normal_dataset, normal_dataset, normal_dataset],ignore_index = True)
normal_aug_dataset['filename'] = normal_aug_file_list
normal_aug_dataset['framerate'] = normal_normal_dataset['framerate'].tolist()
normal_aug_dataset['width'] = normal_normal_dataset['width'].tolist()
normal_aug_dataset['height'] = normal_normal_dataset['height'].tolist()
normal_aug_dataset['frame_count'] = normal_normal_dataset['frame_count'].tolist()
normal_aug_dataset['duration_secs'] = normal_normal_dataset['duration_secs'].tolist()
normal_aug_dataset['disease'] = normal_normal_dataset['disease'].tolist()
normal_aug_dataset['source'] = normal_normal_dataset['source'].tolist()
normal_aug_dataset

,filename,framerate,width,height,frame_count,duration_secs,disease,source
0,20_butterfly_normal_prc_convex_clean_aug_0.avi,15.0,594.0,594.0,142.0,9.47,2,1
1,20_butterfly_normal_prc_convex_clean_aug_1.avi,15.0,820.0,820.0,99.0,6.60,2,1
2,20_butterfly_normal_prc_convex_clean_aug_2.avi,15.0,540.0,540.0,85.0,5.67,2,2
3,20_butterfly_normal_prc_convex_clean_aug_3.avi,15.0,376.0,376.0,60.0,4.00,2,4
4,20_butterfly_normal_prc_convex_clean_aug_4.avi,15.0,450.0,450.0,59.0,3.93,2,4
...,...,...,...,...,...,...,...,...
90,239_paper_normal_prc_convex_clean_aug_0.avi,15.0,600.0,600.0,120.0,8.00,2,8
91,239_paper_normal_prc_convex_clean_aug_1.avi,15.0,886.0,886.0,241.0,16.07,2,9
92,239_paper_normal_prc_convex_clean_aug_2.avi,15.0,436.0,436.0,74.0,4.93,2,9
93,239_paper_normal_prc_convex_clean_aug_3.avi,15.0,482.0,482.0,365.0,24.33,2,9


In [19]:
len(normal_aug_dataset)

95

In [20]:
covid_aug_dataset = pd.DataFrame()
covid_aug_dataset['filename'] = covid_aug_file_list
covid_aug_dataset['framerate'] = covid_dataset['framerate'].tolist()
covid_aug_dataset['width'] = covid_dataset['width'].tolist()
covid_aug_dataset['height'] = covid_dataset['height'].tolist()
covid_aug_dataset['frame_count'] = covid_dataset['frame_count'].tolist()
covid_aug_dataset['duration_secs'] = covid_dataset['duration_secs'].tolist()
covid_aug_dataset['disease'] = covid_dataset['disease'].tolist()
covid_aug_dataset['source'] = covid_dataset['source'].tolist()
covid_aug_dataset

,filename,framerate,width,height,frame_count,duration_secs,disease,source
0,1_butterfly_covid_prc_convex_clean_aug_0.avi,15.0,820.0,820.0,64.0,4.27,0,1
1,2_butterfly_covid_prc_convex_clean_aug_0.avi,15.0,624.0,624.0,158.0,10.53,0,1
2,3_butterfly_covid_prc_convex_clean_aug_0.avi,15.0,1054.0,1054.0,90.0,6.00,0,1
3,4_butterfly_covid_prc_convex_clean_aug_0.avi,15.0,820.0,820.0,108.0,7.20,0,1
4,5_butterfly_covid_prc_convex_clean_aug_0.avi,15.0,810.0,810.0,249.0,16.60,0,1
...,...,...,...,...,...,...,...,...
58,204_paper_covid_prc_convex_clean_aug_0.avi,15.0,680.0,680.0,235.0,15.67,0,8
59,205_paper_covid_prc_convex_clean_aug_0.avi,15.0,680.0,680.0,234.0,15.60,0,8
60,218_clarius_covid_prc_convex_clean_aug_0.avi,15.0,400.0,400.0,104.0,6.93,0,9
61,220_clarius_covid_prc_convex_clean_aug_0.avi,15.0,400.0,400.0,118.0,7.87,0,9


In [21]:
pneumonia_aug_dataset = pd.DataFrame()
pneumonia_pneumonia_dataset = pd.concat([pneumonia_dataset, pneumonia_dataset],ignore_index = True)
pneumonia_aug_dataset['filename'] = pneumonia_aug_file_list
pneumonia_aug_dataset['framerate'] = pneumonia_pneumonia_dataset['framerate'].tolist()
pneumonia_aug_dataset['width'] = pneumonia_pneumonia_dataset['width'].tolist()
pneumonia_aug_dataset['height'] = pneumonia_pneumonia_dataset['height'].tolist()
pneumonia_aug_dataset['frame_count'] = pneumonia_pneumonia_dataset['frame_count'].tolist()
pneumonia_aug_dataset['duration_secs'] = pneumonia_pneumonia_dataset['duration_secs'].tolist()
pneumonia_aug_dataset['disease'] = pneumonia_pneumonia_dataset['disease'].tolist()
pneumonia_aug_dataset['source'] = pneumonia_pneumonia_dataset['source'].tolist()
pneumonia_aug_dataset

,filename,framerate,width,height,frame_count,duration_secs,disease,source
0,25_grepmed_pneumonia_prc_convex_clean_aug_0.avi,15.0,664.0,664.0,300.0,20.00,1,2
1,25_grepmed_pneumonia_prc_convex_clean_aug_1.avi,15.0,344.0,344.0,91.0,6.07,1,2
2,27_grepmed_pneumonia_prc_convex_clean_aug_0.avi,15.0,550.0,550.0,458.0,30.53,1,2
3,27_grepmed_pneumonia_prc_convex_clean_aug_1.avi,15.0,652.0,652.0,187.0,12.47,1,2
4,34_grepmed_pneumonia_prc_convex_clean_aug_0.avi,15.0,540.0,540.0,300.0,20.00,1,2
...,...,...,...,...,...,...,...,...
75,233_clarius_pneumonia_prc_convex_clean_aug_1.avi,15.0,458.0,458.0,482.0,32.13,1,9
76,234_clarius_pneumonia_prc_convex_clean_aug_0.avi,15.0,458.0,458.0,482.0,32.13,1,9
77,234_clarius_pneumonia_prc_convex_clean_aug_1.avi,15.0,458.0,458.0,482.0,32.13,1,9
78,235_clarius_pneumonia_prc_convex_clean_aug_0.avi,15.0,458.0,458.0,482.0,32.13,1,9


In [22]:
video_dataset_copy = pd.concat([video_dataset, normal_aug_dataset,covid_aug_dataset,pneumonia_aug_dataset],ignore_index = True)
video_dataset_copy.reset_index(drop=True)
video_dataset = video_dataset_copy
video_dataset

,filename,framerate,width,height,frame_count,duration_secs,disease,source
0,1_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,64.0,4.27,0,1
1,2_butterfly_covid_prc_convex_clean.avi,15.0,624.0,624.0,158.0,10.53,0,1
2,3_butterfly_covid_prc_convex_clean.avi,15.0,1054.0,1054.0,90.0,6.00,0,1
3,4_butterfly_covid_prc_convex_clean.avi,15.0,820.0,820.0,108.0,7.20,0,1
4,5_butterfly_covid_prc_convex_clean.avi,15.0,810.0,810.0,249.0,16.60,0,1
...,...,...,...,...,...,...,...,...
355,233_clarius_pneumonia_prc_convex_clean_aug_1.avi,15.0,458.0,458.0,482.0,32.13,1,9
356,234_clarius_pneumonia_prc_convex_clean_aug_0.avi,15.0,458.0,458.0,482.0,32.13,1,9
357,234_clarius_pneumonia_prc_convex_clean_aug_1.avi,15.0,458.0,458.0,482.0,32.13,1,9
358,235_clarius_pneumonia_prc_convex_clean_aug_0.avi,15.0,458.0,458.0,482.0,32.13,1,9


In [23]:
video_dataset['disease'].value_counts()

0    126
1    120
2    114
Name: disease, dtype: int64

In [24]:
print(len(glob('data/video/clean_copy2/*')))

481


In [25]:
# separating the target
X = video_dataset
y = video_dataset['disease']

# creating the training and validation set
X_training, X_test, y_training, y_test = train_test_split(X, y, test_size=train_test_ratio, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_training, y_training, shuffle=True, test_size=train_val_ratio, stratify = y_training)

X_train = X_train.reset_index(drop = True)
X_val = X_val.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
y_val = y_val.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [26]:
print("X_train size : {}".format(X_train.shape))
print("y_train size : {}".format(y_train.shape))
print("X_val size : {}".format(X_val.shape))
print("y_val size : {}".format(y_val.shape))
print("X_test size : {}".format(X_test.shape))
print("y_test size : {}".format(y_test.shape))

X_train size : (201, 8)
y_train size : (201,)
X_val size : (51, 8)
y_val size : (51,)
X_test size : (108, 8)
y_test size : (108,)


test 영상만 이후 evaluate 하는 과정에서 frame을 추출하므로 파일 이름을 csv 파일로 저장

In [27]:
# storing the frames from training videos
test_dataset = pd.DataFrame()
test_dataset['filename'] = X_test['filename']
test_dataset.to_csv('data/test_dataset.csv', header=True, index=False)

train 영상과 val 영상은 지금 frame을 추출

다음 과정에서 생성되는 폴더는 코드를 다시 실행하는 경우 중복이나 필요없는 데이터를 제거하기 위해 이 폴더를 삭제하고 진행 (다음 코드 참고)

In [28]:
# create images_14frames_train and images_14frames_val folders, if they do not exist
if not os.path.exists('data/images_captured_train'):
    os.makedirs('data/images_captured_train')
if not os.path.exists('data/images_captured_val'):
    os.makedirs('data/images_captured_val')

files = glob('data/images_captured_train/*')
for f in files:
    os.remove(f)
files = glob('data/images_captured_val/*')
for f in files:
    os.remove(f)

Converts a PIL Image instance to a Numpy array. 
(tf.keras.image.img_to_array와 동일)

In [29]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
	  (im_height, im_width, channels)).astype(np.uint8)
   
def featuring_space(frame_count):
  if frame_count<=28:
    return 1
  elif frame_count<=56:
    return 2
  elif frame_count<=84:
    return 3
  elif frame_count<=112:
    return 4
  else : return 5

**모든 영상에서 뽑아내는 frame 길이를 동일하게 맞춘 뒤에 LSTM을 통과시키고자**

**모든 영상에 대해 frames에 해당하는 프레임을 capture함**

In [30]:
train_label=[]
for i in tqdm(range(X_train.shape[0])):

    count = 0
    current = 0
    videoFile = X_train['filename'][i]
    disease = videoFile.split('_')[2]
    if disease == 'covid' :
      disease_num = 0
    elif disease == 'pneumonia' :
      disease_num = 1
    else : disease_num = 2
    cap = cv2.VideoCapture('data/video/clean_copy2/' + videoFile) # capturing the video from the given path
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    space = featuring_space(frame_count)
    capture_epoch = math.floor( frame_count / (14*space) )
    capture_num = capture_epoch*14

    while(cap.isOpened()):
      if(count==capture_num):
        break
      cap.get(1)
      ret, frame = cap.read()
      if (ret != True):
        break
      if current%space==0 :
        filename = 'data/images_captured_train/' + videoFile + "_frame_%d.jpg" % count; count += 1
        cv2.imwrite(filename, frame)
        if(count%14==0):
          train_label.append(disease_num)
      current=current+1
    cap.release()

len(os.listdir('data/images_captured_train/'))

 51%|█████     | 102/201 [00:48<00:47,  2.11it/s]


KeyboardInterrupt: ignored

In [ ]:
val_label=[]
for i in tqdm(range(X_val.shape[0])):

    count = 0
    current = 0
    videoFile = X_val['filename'][i]
    disease = videoFile.split('_')[2]
    if disease == 'covid' :
      disease_num = 0
    elif disease == 'pneumonia' :
      disease_num = 1
    else : disease_num = 2
    cap = cv2.VideoCapture('data/video/clean_copy2/' + videoFile) # capturing the video from the given path
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    space = featuring_space(frame_count)
    capture_epoch = math.floor( frame_count / (14*space) )
    capture_num = capture_epoch*14

    while(cap.isOpened()):
      if(count==capture_num):
        break
      cap.get(1)
      ret, frame = cap.read()
      if (ret != True):
        break
      if current%space==0 :
        filename = 'data/images_captured_val/' + videoFile + "_frame_%d.jpg" % count; count += 1
        cv2.imwrite(filename, frame)
        if(count%14==0):
          val_label.append(disease_num)
      current=current+1
    cap.release()

len(os.listdir('data/images_captured_val/'))

In [ ]:
X_val

In [ ]:
train_images = glob("data/images_captured_train/*.jpg")

train_image_name = []
train_image_class = []
for i in tqdm(range(len(train_images))):
    # creating the image name
    train_image_name.append(train_images[i].split('/')[2])
    # creating the class of image
    disease = train_images[i].split('/')[2].split('_')[2]
    if disease == 'covid':
        train_image_class.append(0)
    elif disease == 'pneumonia':
        train_image_class.append(1)
    else:
        train_image_class.append(2)

train_image_dataset = pd.DataFrame()
train_image_dataset['filename'] = train_image_name
train_image_dataset['disease'] = train_image_class # 

train_image_dataset.to_csv('data/image_files_clean_train.csv', header=True, index=False)

In [ ]:
train_image_dataset

In [ ]:
val_images = glob("data/images_captured_val/*.jpg")

val_image_name = []
val_image_class = []
for i in tqdm(range(len(val_images))):
    # creating the image name
    val_image_name.append(val_images[i].split('/')[2])
    # creating the class of image
    disease = val_images[i].split('/')[2].split('_')[2]
    if disease == 'covid':
        val_image_class.append(0)
    elif disease == 'pneumonia':
      val_image_class.append(1)
    else:
        val_image_class.append(2)

val_image_dataset = pd.DataFrame()
val_image_dataset['filename'] = val_image_name
val_image_dataset['disease'] = val_image_class 

val_image_dataset.to_csv('data/image_files_clean_val.csv', header=True, index=False)

In [ ]:
val_image_dataset

In [ ]:
# creating an empty list
train_images = []

# for loop to read and store frames
for i in tqdm(range(train_image_dataset.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('data/images_captured_train/'+train_image_dataset['filename'][i], target_size=(height,width,channels))

    # converting it to array
    img_np = image.img_to_array(img)

    # normalizing the pixel value
    img_np = img_np / 255

    # appending the image to the images list
    train_images.append(img_np)
    
# converting the list to numpy array
X_train = np.array(train_images)
print(X_train.shape)

In [ ]:
# creating an empty list
val_images = []

# for loop to read and store frames
for i in tqdm(range(val_image_dataset.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('data/images_captured_val/'+val_image_dataset['filename'][i], target_size=(height,width,channels))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the images list
    val_images.append(img)
    
# converting the list to numpy array
X_val = np.array(val_images)
print(X_val.shape)

In [ ]:
X_train=X_train.reshape(-1,14,224,224,3)
X_val=X_val.reshape(-1,14,224,224,3)
train_label = np.array(train_label)
val_label = np.array(val_label)
print(X_train.shape)
print(X_val.shape)
print(train_label.shape)
print(val_label.shape)
print(X_test.shape)

In [ ]:
np.save('data/X_train_save_0', X_train)
np.save('data/X_val_save_0', X_val)
np.save('data/y_train_save_0', train_label)
np.save('data/y_val_save_0', val_label)